# The report is written in the other file (Bitmask.ipynb)

In [36]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy
import sys, time

In [37]:
class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

        trnsps = self.board.T
        anti_trnsps = np.fliplr(self.board).T

        self.variants = [
            self.board,
            # trnsps,
            # anti_trnsps,
            
            # np.rot90(self.board, k=1),
            # np.rot90(self.board, k=2),
            # np.rot90(self.board, k=3),

            # np.rot90(trnsps, k=1),
            # np.rot90(trnsps, k=2),
            # np.rot90(trnsps, k=3),

            # np.rot90(anti_trnsps, k=3),
            # np.rot90(anti_trnsps, k=3),
            # np.rot90(anti_trnsps, k=3),
            
        ]

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)

    def __hash__(self):
        return min(hash(tuple(map(tuple, variant))) for variant in self.variants)
    
    def __eq__(self, other):
        if not isinstance(other, LightsOutPuzzle):
            return False        
        return any(np.array_equal(variant, other.board) for variant in self.variants)
    
    def __lt__(self, other):
        if not isinstance(other, LightsOutPuzzle):
            return NotImplemented
        return np.sum(self.board) < np.sum(other.board)


In [38]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

In [39]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [40]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    create_random_board(5, 42),
    create_random_board(5, 41),
    create_random_board(5, 42, 5),
]

In [41]:
def bfs_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    queue = collections.deque()
    queue.append([puzzle, []])

    visited = set()
    visited.add(hash(puzzle))
    cnt = 0
    while queue:
        curr, mvs = queue.popleft()
        if curr.is_solved(): return (mvs, cnt)

        for mv in puzzle.get_moves():
            pzl = LightsOutPuzzle(deepcopy(curr.board))
            pzl.toggle(*mv)

            if hash(pzl) not in visited:
                visited.add(hash(pzl))
                cnt += 1
                queue.append((pzl, mvs + [mv]))
    return (None, cnt)

pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[4]))
print(pzl)
s = time.time()
print(bfs_solve(pzl))
print(time.time() - s)

0 0 0 1
0 1 1 0
0 1 1 0
1 1 1 0
([(0, 3), (1, 2), (3, 1)], 1749)
0.0861973762512207


In [42]:
def ids_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    cnt = 0
    def dls(puzzle : LightsOutPuzzle, dpth : int, path : list[tuple[int, int]]) -> list[tuple[int, int]]:
        nonlocal cnt
        if puzzle.is_solved(): return path

        if dpth == 0: return None

        for mv in puzzle.get_moves():
            pzl = LightsOutPuzzle(puzzle.board.copy().tolist())
            pzl.toggle(*mv)
            cnt += 1
            if (pth := dls(pzl, dpth - 1, path + [mv])) is not None: return pth
        return None

    mx_dpth = 0
    while mx_dpth < puzzle.size * puzzle.size:
        path = dls(puzzle, mx_dpth, [])
        if path is not None: return (path, cnt)
        mx_dpth += 1
    return (None, cnt)

pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[4]))
print(pzl)
s = time.time()
print(ids_solve(pzl))
print(time.time() - s)

0 0 0 1
0 1 1 0
0 1 1 0
1 1 1 0
([(0, 3), (1, 2), (3, 1)], 1225)
0.010907411575317383


In [43]:
def heuristic1(puzzle : LightsOutPuzzle):
  return np.sum(puzzle.board)

def heuristic2(puzzle : LightsOutPuzzle):
  return heuristic1(puzzle) / 5 

def manhattan_distance_heuristic(puzzle : LightsOutPuzzle):
  h = 0
  for x in range(puzzle.size):
    for y in range(puzzle.size):
      if puzzle.board[x, y] == 1:
        h += (x + y)
  return h

heuristics = [heuristic1, heuristic1, manhattan_distance_heuristic]

In [50]:
def weighted_heuristic1(puzzle: LightsOutPuzzle , alpha = 5):
  return heuristic1(puzzle) * alpha

def weighted_heuristic2(puzzle : LightsOutPuzzle, alpha = 5):
  return heuristic2(puzzle) * alpha

def weighted_heuristic3(puzzle : LightsOutPuzzle, alpha = 5):
  return manhattan_distance_heuristic(puzzle) * alpha 

weighted_heuristics = [weighted_heuristic1, weighted_heuristic2, weighted_heuristic1]
weights = [2, 5, 10]

In [51]:
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int], cnt_flag = False) -> Tuple[List[Tuple[int, int]], int]:
    open_set = []
    heapq.heappush(open_set, (0, 0, puzzle, []))
    g_score = {puzzle: 0}
    f_score = {puzzle: heuristic(puzzle)}
    cnt = 0

    while open_set:
        # if cnt_flag and cnt % 1000 == 0:
        #     sys.stdout.write(f"\rCurrent cnt: {cnt}")
        #     sys.stdout.flush()
        _, _, current, path = heapq.heappop(open_set)

        if current.is_solved():
            return (path, cnt)

        for move in current.get_moves():
            neighbor = LightsOutPuzzle(current.board.tolist())
            neighbor.toggle(*move)

            gnxt = g_score[current] + 1
            if neighbor not in g_score or gnxt < g_score[neighbor]:
                g_score[neighbor] = gnxt
                f_score[neighbor] = g_score[neighbor] + heuristic(neighbor)
                heapq.heappush(open_set, (f_score[neighbor], gnxt, neighbor, path + [move]))
                cnt += 1

    return (None, cnt)


pzl = LightsOutPuzzle([
    [0, 1, 1, 0],
    [1, 0, 0, 1],
    [0, 1, 1, 0],
    [0, 0, 0, 0]
])
pzl = LightsOutPuzzle(deepcopy(tests[-1]))

print(pzl, end="\n\n")
start_time = time.time()
p, c = astar_solve(pzl, manhattan_distance_heuristic, cnt_flag = True)
print("\nPath:", p)
print("Nodes Visited:", c)
print(f"Time taken: {time.time() - start_time:.4f} seconds")

pzl = deepcopy(pzl)
for mv in p:
    pzl.toggle(*mv)
print(f"Is solved: {pzl.is_solved()}", end='\n\n')
print(pzl, end='\n\n')

1 0 0 0 0
0 0 1 0 0
1 0 1 0 0
0 1 0 0 0
0 0 0 0 0


Path: [(2, 1), (1, 1), (0, 0)]
Nodes Visited: 72
Time taken: 0.0044 seconds
Is solved: True

0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0
0 0 0 0 0



In [52]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [53]:
def run_searches(func: Callable, args: Any, time_limit: float = 60.0, name: str = "") -> SearchResult:
    result = []

    def target():
        try:
            solution, nodes_visited = func(*args)
            result.append((solution, nodes_visited))
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()

    start_time = time.time()
    thread.join(timeout=time_limit)

    if thread.is_alive() or not result:
        print(f"\nTime limit of {time_limit} seconds exceeded for {name}")
        return SearchResult(SearchStatus.TIMEOUT)

    if isinstance(result[0], Exception):
        raise result[0]

    solution, nodes_visited = result[0]
    show_solution(args[0], solution, name, nodes_visited)
    time_taken = time.time() - start_time
    return SearchResult(SearchStatus.SOLVED if solution else SearchStatus.NO_SOLUTION, solution, nodes_visited, time_taken)

In [54]:
def run_test(test: List, heuristics: List[Callable[[Any], int]], time_limit: float = 60.0):

    print(f"Running test:\n {test}")

    puzzle = LightsOutPuzzle(test)

    bfs_result = run_searches(bfs_solve, (deepcopy(puzzle),), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, (deepcopy(puzzle),), name="IDS", time_limit=time_limit)

    astar_results = []
    for heuristic in heuristics:
        astar_results.append(
            run_searches(
                astar_solve,
                (deepcopy(puzzle), heuristic),
                name=f"A*({heuristic.__name__})",
                time_limit=time_limit,
            )
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
          weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, alpha = weight)),
                  name=f"weighted A*({heuristic.__name__}, alpha = {weight})",
                  time_limit=time_limit,
              )
          )

    print("\n-------------------------------\n")

    return (
        test,
        bfs_result,
        ids_result,
        *astar_results,
        *weighted_astar_results
    )

In [55]:
results: List[Tuple[str, SearchResult, SearchResult, SearchResult, SearchResult]] = []

for test in tests:
    results.append(run_test(test, heuristics, 120))

Running test:
 [[1 1 1]
 [1 1 1]
 [1 0 0]]

Solving with BFS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 94

Solving with IDS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 34

Solving with A*(heuristic1):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with A*(heuristic1):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with A*(manhattan_distance_heuristic):
Solution: [(0, 2), (1, 0)]
Nodes visited: 17

Solving with weighted A*(weighted_heuristic1, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic1, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic1, alpha = 10):
Solution: [(1, 0), (0, 2)]
Nodes visited: 38

Solving with weighted A*(weighted_heuristic2, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with weighted A*(weighted_heuristic2, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 24

Solving with weighted A*(weighted_heuristic2, alpha = 10):
Solut

In [56]:
table_data: List[List[str]] = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),
            str(result[1]),
            str(result[2]),
        ]
    )

    for i in range(len(heuristics)):
        table_data[-1].append(str(result[2 + i]))

    for i in range(len(weighted_heuristics)):
        for j in range(len(weights)):
          table_data[-1].append(str(result[2 + len(heuristics) + i * 2 + j]))

    table_data.append("\n\n\n")

if table_data[-1] == "\n\n\n":
    table_data.pop()

print(len(table_data))
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS",
            "IDS",
            *[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            *[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="grid",
    )
)

17
Results:
+-----------+----------------------------------------------------+----------------------------------------------------+----------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------